In [1]:
import tensorflow as tf
# Fixes bad convolution
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
@tf.function
def solve(s):
    s = tf.strings.split(s, '\n\n')
    s = tf.strings.split(s, '\n')
    s = s.to_tensor()
    
    tile_ids = s[:,0]
    tile_ids = tf.strings.regex_replace(tile_ids, '[^0-9]', '')
    tile_ids = tf.strings.to_number(tile_ids, out_type=tf.float64)
    
    tiles = s[:,1:]
    tiles = tf.strings.bytes_split(tiles).to_tensor()
    tiles = tf.where(tiles == '.', 0.0, 1.0)

    top_rows = tiles[:,0]
    bottom_rows = tiles[:,-1]
    left_col = tiles[:,:,0]
    right_col = tiles[:,:,-1]
    
    all_edges = tf.stack((top_rows, bottom_rows, left_col, right_col), axis=1)
    rev_all_edges = all_edges[...,::-1]
    
    all_edges = tf.concat((all_edges, rev_all_edges), axis=1)
    
    ae_shape = tf.shape(all_edges)
    rnge = tf.range(ae_shape[0])
    inner_rnge = tf.range(ae_shape[1])
    res = tf.constant(1, tf.float64)
    for i in rnge:
        mask = (rnge == i)
        cur_edges = all_edges[mask][0]
        other_edges = all_edges[~mask]
        n = tf.constant(0, tf.int64)
        for j in inner_rnge:
            n += tf.math.count_nonzero(tf.reduce_all((cur_edges[j] == other_edges), axis=-1))
        if n == 4:
            res *= tile_ids[i]
    
    return res

%timeit solve(tf.io.read_file('day20.txt'))

131 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
solve(tf.io.read_file('day20.txt'))

<tf.Tensor: shape=(), dtype=float64, numpy=104831106565027.0>

In [4]:
def print_final_img(final_image):   
    for i in range(final_image.shape[0]):
        for k in range(final_image.shape[2]):
            for j in range(final_image.shape[1]):
                img = final_image[i,j]
                for l in range(final_image.shape[3]):
                    if img[k,l] == 1:
                        print('#', end='')
                    else:
                        print('.', end='')
                print(' ', end='')
            print()
        print()

@tf.function
def solve(s):
    s = tf.strings.split(s, '\n\n')
    s = tf.strings.split(s, '\n')
    s = s.to_tensor()
    
    tile_ids = s[:,0]
    tile_ids = tf.strings.regex_replace(tile_ids, '[^0-9]', '')
    tile_ids = tf.strings.to_number(tile_ids, out_type=tf.float64)
    
    tiles = s[:,1:]
    tiles = tf.strings.bytes_split(tiles).to_tensor()
    tiles = tf.where(tiles == '.', 0.0, 1.0)
    tiles_shape = tf.shape(tiles)

    top_rows = tiles[:,0]
    bottom_rows = tiles[:,-1]
    left_col = tiles[:,:,0]
    right_col = tiles[:,:,-1]
    
    all_edges = tf.stack((top_rows, bottom_rows, left_col, right_col), axis=1)
    rev_all_edges = all_edges[...,::-1]
    
    all_edges = tf.concat((all_edges, rev_all_edges), axis=1)
    
    ae_shape = tf.shape(all_edges)
    
    # Find the length of a side of the square
    side_length = tf.cast(tf.sqrt(tf.cast(ae_shape[0], tf.float32)), tf.int32)

    rnge = tf.range(ae_shape[0])
    inner_rnge = tf.range(ae_shape[1])
    res = tf.constant(1, tf.float64)
    
    lu_corner = tf.zeros_like(tiles[0])
    remaining_tiles_mask = tf.fill((ae_shape[0],), True)
    
    # First we try to find the left upper tile (without rotation or flipping)
    for i in rnge:
        mask = (rnge == i)
        cur_edges = all_edges[mask][0]
        other_edges = all_edges[~mask]
        n = tf.constant(0, tf.int64)
        for j in inner_rnge:
            n += tf.math.count_nonzero(tf.reduce_all((cur_edges[j] == other_edges), axis=-1))
        
        # 4 matches means 2 sides match (doubled because of symmetry)
        if n == 4:
            lu_corner = tiles[i]
            
            # Do we have a match on the top row?
            match_top = tf.reduce_any(tf.reduce_all((cur_edges[0] == other_edges), axis=-1))
            # Do we have a match on the left column?
            match_left = tf.reduce_any(tf.reduce_all((cur_edges[2] == other_edges), axis=-1))
            
            if match_top:
                lu_corner = lu_corner[::-1]
            
            if match_left:
                lu_corner = lu_corner[:,::-1]

            remaining_tiles_mask = tf.tensor_scatter_nd_update(remaining_tiles_mask, [[i]], [False])
            break
    
    side_rnge = tf.range(side_length)
    
    # Final image will be a grid of tiles
    final_image = tf.zeros(tf.concat(([side_length], [side_length], tiles_shape[1:]), 0))
    final_image = tf.tensor_scatter_nd_update(final_image, [[0, 0]], [lu_corner])

    for i in side_rnge:
        for j in side_rnge:
            if i == 0 and j == 0:
                continue
            edges = all_edges[remaining_tiles_mask]

            if j > 0:
                last_tile = final_image[i,j-1]
                last_right_column = last_tile[:,-1]
            else:
                last_tile = final_image[i-1,j]
                last_right_column = last_tile[-1]
            
            where = tf.where(tf.reduce_all(last_right_column == edges, axis=-1))
            matching_tile_idx = where[0,0]                    
            transformation_idx = where[0,1]
            
            tile = tiles[remaining_tiles_mask][matching_tile_idx]
            
            remaining_idx_to_update = tf.where(remaining_tiles_mask)[matching_tile_idx][0]
            remaining_tiles_mask = tf.tensor_scatter_nd_update(remaining_tiles_mask, [[remaining_idx_to_update]], [False])

            #print(transformation_idx)
            # Match top row
            if transformation_idx == 0:
                tile = tf.image.rot90(tile[...,None], k=3)[...,0]
                tile = tile[:,::-1]
            # Match bottom row
            elif transformation_idx == 1:
                tile = tf.image.rot90(tile[...,None], k=3)[...,0]
            # Match right col (we skip left col because then tile is already match)
            elif transformation_idx == 3:
                tile = tile[:,::-1]
            # Top reversed
            elif transformation_idx == 4:
                tile = tf.image.rot90(tile[...,None], k=1)[...,0]
            # Bottom reversed
            elif transformation_idx == 5:
                tile = tf.image.rot90(tile[...,None], k=3)[...,0]
                tile = tile[::-1]
            # Left reversed
            elif transformation_idx == 6:
                tile = tile[::-1]
            # Right reversed
            elif transformation_idx == 7:
                tile = tile[::-1,::-1]
            
            if j == 0:
                tile = tf.image.rot90(tile[...,None], k=1)[...,0]
                tile = tile[::-1]

            
            final_image = tf.tensor_scatter_nd_update(final_image, [[i, j]], [tile])

    # Remove borders from image
    final_image = final_image[:,:,1:-1,1:-1]
    
    ta = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
    
    for i in tf.range(tf.shape(final_image)[0]):
        subta = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
        for j in tf.range(tf.shape(final_image)[1]):
            subta = subta.write(j, tf.transpose(final_image[i,j]))
        cc = tf.transpose(subta.concat())
        ta = ta.write(i, cc)
    
    final_image = ta.concat()
        
    kernel = '                  # \n'\
      +'#    ##    ##    ###\n'\
      +' #  #  #  #  #  #   '
    kernel = tf.where(tf.strings.bytes_split(tf.strings.split(kernel, '\n')).to_tensor() == '#', 1.0, 0.0)
    kernel = tf.concat((tf.zeros((3, 3)), kernel), 1)[...,None]
    n_spaces_monster = tf.reduce_sum(kernel)
    # Make upside down dragon as well
    kernel = tf.stack((kernel, kernel[::-1]), -1)
    
    final_images = tf.stack([tf.image.rot90(final_image[...,None], k=i) for i in range(4)], 0)
    convolution = tf.nn.conv2d(final_images, kernel, strides=[1, 1], padding="SAME")
    n_monsters = tf.math.count_nonzero(convolution == n_spaces_monster, dtype=tf.float32)
    return tf.reduce_sum(final_image) - (n_monsters * n_spaces_monster)

%timeit solve(tf.io.read_file('day20.txt'))

549 ms ± 8.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
solve(tf.io.read_file('day20.txt'))

<tf.Tensor: shape=(), dtype=float32, numpy=2093.0>